## Question(s)
* Are batters paid more than pitchers?
* Is there a correlation between performance and pay?
* Do teams with high-performing pitchers do better than teams with high-performing batters or vice versa?

In [11]:
# import datadotworld to get baseball data
import datadotworld as dw
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import pearsonr
import requests, zipfile, io, os, calendar
from datetime import datetime 
import glob # used for searching for files
from easymoney.money import EasyPeasy # easy money will normalize salaries for inflation
ep = EasyPeasy() # Create an instance of the EasyPeasy Class
import wbdata # World Bank data required for easy money

from __future__ import print_function
%matplotlib inline
import seaborn as sns 
sns.set(style='ticks')

In [25]:
# Import from data.world so no local files needed (also faster)
baseball = dw.load_dataset('natereed/lahman-baseball-data')

# get an initial look at files includes in repo
baseball.describe()

baseball_master = dw.query('natereed/lahman-baseball-data', 'SELECT * FROM master LIMIT 10')
baseball_master.dataframe

baseball_teams = dw.query('natereed/lahman-baseball-data', 'SELECT * FROM teams LIMIT 10')
baseball_teams.dataframe

baseball_batting = dw.query('natereed/lahman-baseball-data', 'SELECT * FROM batting LIMIT 10')
baseball_batting.dataframe

baseball_pitching = dw.query('natereed/lahman-baseball-data', 'SELECT * FROM pitching LIMIT 10')
baseball_pitching.dataframe

playerid  yearid  stint teamid lgid   w   l   g  gs  cg  ...    ibb   wp  \
0  bechtge01    1871      1    PH1   NA   1   2   3   3   2  ...   None  NaN   
1  brainas01    1871      1    WS3   NA  12  15  30  30  30  ...   None  NaN   
2  meyerle01    1871      1    PH1   NA   0   0   1   0   0  ...   None  NaN   
3  fleetfr01    1875      2    BR2   NA   0   1   2   1   1  ...   None  NaN   
4  glassja01    1887      1    IN3   NL   0   0   1   0   0  ...   None  0.0   
5  cascajo01    1936      1    BOS   AL   0   2  10   1   0  ...   None  0.0   
6  castlsl01    1936      1    NY1   NL   4   7  29  12   2  ...   None  4.0   
7  chaplti01    1936      1    BSN   NL  10  15  40  31  14  ...   None  5.0   
8  chaseke01    1936      1    WS1   AL   0   0   1   0   0  ...   None  0.0   
9  cheliit01    1936      1    CHA   AL   4   3  18   6   5  ...   None  0.0   

   hbp  bk     bfp   gf    r    sh    sf  gidp  
0  NaN   0     NaN  NaN   42  None  None  None  
1  NaN   0     NaN  NaN  292  None  None  None  
2  NaN   0     NaN  NaN    1  None  None  None  
3  NaN   0     NaN  NaN   20  None  None  None  
4  0.0   0     3.0  1.0    0  None  None  None  
5  0.0   0    91.0  5.0   16  None  None  None  
6  5.0   0   529.0  9.0   80  None  None  None  
7  3.0   0  1003.0  6.0  131  None  None  None  
8  0.0   0    12.0  0.0    3  None  None  None  
9  0.0   0   378.0  6.0   51  None  None  None  

[10 rows x 30 columns]

### The following cell includes an optional alternative method for accessing the data by downloading a zip file from SeanLahman.com
* if the previous cell was successfully run, there is no need to run this cell

In [ ]:
# dir_name = '' # set directory for zip file if different than current
# change directories into extract directory
# os.chdir(dir_name)

# if no directory specified, zip file will go to current working directory
dir_name = os.getcwd()

url = 'http://seanlahman.com/files/database/baseballdatabank-2017.1.zip'

def download_file(url):
    print("downloading zip file from", url)
    local_filename = url.split('/')[-1]
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    print("file downloaded to",dir_name)
    return local_filename

# retrieve baseball data
download_file(url)

zip_ref = zipfile.ZipFile(dir_name + '\\baseballdatabank-2017.1.zip', 'r')
zip_dir = dir_name + "\lahman_baseball"
zip_ref.extractall(zip_dir)
print("extracting zip file to", zip_dir)
zip_ref.close()
print("files extracted to",zip_dir)
print('finished processing at', str(datetime.now().hour) + ':' + str(datetime.now().minute))

# list out the files you just extracted
os.chdir(dir_name + '\lahman_baseball\baseballdatabank-2017.1\core')
%ls

### Read in data of interest as dataframes and review field names

In [9]:
# data.world has their own built-in dataframe function, "dataframes"
master = baseball.dataframes['master']
#print('The master df has ' + str(master.shape[0]) + ' rows and ' + str(master.shape[1]) + ' columns')
master.describe(include='all')

teams = baseball.dataframes['teams']
#print('The teams df has ' + str(teams.shape[0]) + ' rows and ' + str(teams.shape[1]) + ' columns')

pitching = baseball.dataframes['pitching']
#print('The pitching df has ' + str(pitching.shape[0]) + ' rows and ' + str(pitching.shape[1]) + ' columns')

batting = baseball.dataframes['batting']
#print('The batting df has ' + str(batting.shape[0]) + ' rows and ' + str(batting.shape[1]) + ' columns')

salaries = baseball.dataframes['salaries']
#print('The salaries df has ' + str(salaries.shape[0]) + ' rows and ' + str(salaries.shape[1]) + ' columns')

hall_of_fame = baseball.dataframes['halloffame']
#print('The hall of fame df has ' + str(hall_of_fame.shape[0]) + ' rows and ' + str(hall_of_fame.shape[1]) + ' columns')

# salaries only available from 1985 on
teams = teams[teams['yearid'] >= 1985]

print('Master\n',master.head(10))
print('Teams\n',teams.head(10))
print('Pitching\n',pitching.head(10))
print('Batting\n',batting.head(10))
print('Salaries\n',salaries.head(10))
print('Hall of Fame\n',hall_of_fame.head(10))

# Retain only Year, Team, Rank, Runs scored, Opponents runs scored, Games played, Wins, Hits by batters, Walks by batters, 
# Batters hit by pitch, At bats, Sacrifice flies, Homeruns by batters, Doubles, Triples
#teams = teams[['yearid', 'teamid', 'rank', 'r', 'ra', 'g', 'w', 'h', 'bb', 'hbp', 'ab', 'sf', 'hr', '2b', '3b']]
#teams = teams.set_index(['yearid', 'teamid'])
#teams.head()

Master
     playerid  birthyear  birthmonth  birthday birthcountry birthstate  \
0  aardsda01     1981.0        12.0      27.0          USA         CO   
1  aaronha01     1934.0         2.0       5.0          USA         AL   
2  aaronto01     1939.0         8.0       5.0          USA         AL   
3   aasedo01     1954.0         9.0       8.0          USA         CA   
4   abadan01     1972.0         8.0      25.0          USA         FL   
5   abadfe01     1985.0        12.0      17.0         D.R.  La Romana   
6  abadijo01     1854.0        11.0       4.0          USA         PA   
7  abbated01     1877.0         4.0      15.0          USA         PA   
8  abbeybe01     1869.0        11.0      11.0          USA         VT   
9  abbeych01     1866.0        10.0      14.0          USA         NE   

      birthcity  deathyear  deathmonth  deathday     ...     weight height  \
0        Denver        NaN         NaN       NaN     ...      220.0   75.0   
1        Mobile        NaN      

## Data Wrangling

In [55]:
# get salaries by year
salaries_by_year = salaries.groupby(['yearid','teamid'])['salary'].sum()
salaries_by_year.head()

yearid  teamid
1985    ATL       14807000
        BAL       11560712
        BOS       10897560
        CAL       14427894
        CHA        9846178
Name: salary, dtype: int64

In [53]:
# teams df with wins by year
teams_with_wins = teams.loc[:, ['yearid', 'teamid', 'w']]
teams_with_wins.head()

yearid  teamid   w
yearid teamid                    
1871   BS1        NaN     NaN  20
       CH1        NaN     NaN  19
       CL1        NaN     NaN  10
       FW1        NaN     NaN   7
       NY2        NaN     NaN  16

## Data Exploration

## Draw Conclusions

In [75]:
# hall of fame players who were inducted
hall = set(hall_of_fame[(hall_of_fame.inducted=='true') & (hall_of_fame.category=='Player')].player_id)

C:\Users\brocd8s\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


TypeError: invalid type comparison

## Communicate Findings